# Creating a Discussion

The simplest task you can accomplish with this library is to create a small discussion between LLMs. 

This guide will teach you the basic setup of the library. You will understand how to setup models, user-agents and how to coordinate them in a discussion. By the end of htis guide, you will be able to run a small discussion with a moderator and save it to the disk for persistence.

## Basics

### The Model

SynDisco can theoretically support any LLM, as long as it is wrapped in a `BaseModel` wrapper. The `BaseModel` class is a very simple interface with one method. This method gives the underlying LLM input, and returns its output to the library.

There already exists a `TransformersModel` class which handles models from the `transformers` python library. In 90% of your applications, this will be enough. We can load a TransformersModel using the following code:

In [1]:
from syndisco.model import TransformersModel

llm = TransformersModel(
    model_path="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    name="test_model",
    max_out_tokens=100,
)

This will download a small LLM from huggingface. You can substitute the model_path for any similar model in [HuggingFace](https://huggingface.co/) supporting the Transformers library.

### Creating personas

All `actors` can be defined by a `persona`, aka a set of attributes that define them. These attributes can be age, ethnicity, and even include special instructions on how they should behave.

Creating a persona programmatically is simple:

In [2]:
from syndisco.actors import Persona

persona_data = [
    {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "special_instructions": "",
        "personality_characteristics": [
            "compassionate",
            "patient",
            "diligent",
            "overwhelmed",
        ],
    },
    {
        "username": "Giannis",
        "age": 21,
        "sex": "male",
        "education_level": "College",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Game Developer",
        "special_instructions": "",
        "personality_characteristics": [
            "strategic",
            "meticulous",
            "nerdy",
            "hyper-focused",
        ],
    },
]

personas = [Persona(**data) for data in persona_data]

for persona in personas:
    print(persona)

{"username": "Emma35", "age": 38, "sex": "female", "sexual_orientation": "Heterosexual", "demographic_group": "Latino", "current_employment": "Registered Nurse", "education_level": "Bachelor's", "special_instructions": "", "personality_characteristics": ["compassionate", "patient", "diligent", "overwhelmed"]}
{"username": "Giannis", "age": 21, "sex": "male", "sexual_orientation": "Pansexual", "demographic_group": "White", "current_employment": "Game Developer", "education_level": "College", "special_instructions": "", "personality_characteristics": ["strategic", "meticulous", "nerdy", "hyper-focused"]}


Since creating a lot of distinct users is essential in running large-scale experiments, users are usually defined in JSON format. That way, you can change anything without touching your code!

[Here](https://github.com/dimits-ts/synthetic_moderation_experiments/blob/master/data/discussions_input/personas/personas.json) is an applied example of how to mass-define user personas through JSON files.

### Creating the user-agents

Having a `persona` and a `model` we can finally create an `actor`. The actor will personify the selected persona using the model to talk.

Besides a persona and a model, the actors will also need instructions and a context. By convention, all actors share the same context, and all user-agents share the same instructions. Personalized instructions are defined in the actor's persona.

In [3]:
from syndisco.actors import Actor, ActorType


CONTEXT = "You are taking part in an online conversation"
INSTRUCTIONS = "Act like a human would"

actors = [
    Actor(
        model=llm,
        persona=p,
        actor_type=ActorType.USER,
        context=CONTEXT,
        instructions=INSTRUCTIONS
    )
    for p in personas
]

### Managing turn-taking

In real-life discussions, who gets to speak at each point in time is determined by complex social dynamics, which are difficult to realistically model.
However, there are ways with which we can simulate these dynamics. 

SynDisco uses the `TurnManager` class to model turn taking. Two implementations are available by default: Round Robin, and Random Weighted

- `Round Robin` is the simplest, most intuitive way to model turn-taking; everyone gets to talk once per round. Once everyone talks once, they get to talk again in the same sequence

In [4]:
from syndisco.turn_manager import RoundRobin


rrobin_turn_manager = RoundRobin(["John", "Mary", "Howard", "Todd"])

for i in range(10):
    print(next(rrobin_turn_manager))

John
Mary
Howard
Todd
John
Mary
Howard
Todd
John
Mary


- `RandomWeighted` on the other hand throws a weighted coin on each round. If the coin flip succedes, the previous user gets to respond. If not, another user is selected at random

In [5]:
from syndisco.turn_manager import RandomWeighted


rweighted_turn_manager = RandomWeighted(
    names=["John", "Mary", "Howard", "Todd"], p_respond=0.5
)

for i in range(10):
    print(next(rweighted_turn_manager))

Mary
John
Howard
Mary
Todd
Mary
Todd
Howard
Todd
Mary


## Generating a discussion

Let's start with the most basic task; a single discussion between two user-agents.

Since we only have two users, a RoundRobin approach where each user takes a turn sequentially is sufficient.

Now we can run a simple discussion

In [6]:
from syndisco.jobs import Discussion


turn_manager = RoundRobin([actor.get_name() for actor in actors])
conv = Discussion(next_turn_manager=turn_manager, users=actors)
conv.begin()

  0%|          | 0/5 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


User Emma35 posted:
"Hey everyone, I just got back from a long shift at the hospital and
I'm exhausted. Anyone else have days like that? I swear, some days I
feel like I'm running on coffee fumes and adrenaline alone" 

User Giannis posted:
{"role": "Giannis", "content": "Hey Emma, sorry to hear that. I feel
you. I've had those days too, especially when I'm working on a tight
deadline for a game. The adrenaline rush from the initial project
start can be replaced by a crushing sense of responsibility once the
clock starts ticking. What do you do to recharge after a long shift?"} 

User Emma35 posted:
{"role": "Emma35", "content": "Giannis, I totally get what you mean!
As a nurse, I'm always on call and have to be ready to jump into
action at a moment's notice. But when I'm not working, I love to
unwind with a good book and a cup of tea. Sometimes I'll take a long
bath, light some candles, and just relax. I also try to prioritize
self-care, whether that's getting a massage, taking a yoga

Let's add a moderator to oversee the discussion.

In [7]:
MODERATOR_INSTRUCTIONS = "You are a moderator. Oversee the discussion"

moderator_persona = Persona(
    **{
        "username": "Moderator",
        "age": 41,
        "sex": "male",
        "education_level": "PhD",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Moderator",
        "special_instructions": "",
        "personality_characteristics": [
            "strict",
            "neutral",
            "just",
        ],
    }
)

moderator = Actor(
    model=llm,
    persona=moderator_persona,
    actor_type=ActorType.USER,
    context=CONTEXT,
    instructions=MODERATOR_INSTRUCTIONS
)

# remember to update this!
turn_manager = RoundRobin(
    [actor.get_name() for actor in actors] + [moderator.get_name()]
)
conv = Discussion(
    next_turn_manager=turn_manager,
    users=actors + [moderator],
    moderator=moderator,
)
conv.begin()

  0%|          | 0/5 [00:00<?, ?it/s]

User Emma35 posted:
"Hey everyone, I just got back from a long shift at the hospital and
I'm exhausted. Anyone else have days like that? I swear, some days I
feel like I'm running on coffee fumes and adrenaline alone" 

User Moderator posted:
{"role": "Moderator", "content": "Hey Emma35, sorry to hear that
you're feeling exhausted. It's completely understandable, especially
after a long shift at the hospital. Many healthcare workers put in
long hours and sacrifice a lot to care for their patients. How do you
manage to cope with the physical and emotional demands of your job? Do
you have any self-care routines or strategies that help you
recharge?"} 

User Giannis posted:
"Hey Emma35, I totally feel you. As a game developer, I'm used to long
hours of focus and concentration, but I can imagine how draining it
must be to deal with the emotional toll of working in a hospital. I've
found that for me, taking short breaks to stretch and move around
really helps. I also try to prioritize my sl

### Saving a discussion to the disk

It's best practice to save the results of each discussion after it has concluded. This way, no matter what happens to the program executing the discussions, progress will be checkpointed.

The `Discussion` class provides a method for saving its logs and related metadata to a JSON file:

In [8]:
import tempfile
import json

tp = tempfile.NamedTemporaryFile(delete=True)

conv.to_json_file(tp.name)

# if you are running this on Windows, uncomment this line
# tp.close()
with open(tp.name, mode="rb") as f:
    print(json.dumps(json.load(f), indent=2))

{
  "id": "4c2cc6c2-b73c-4da2-ae36-71ed0b0e6565",
  "timestamp": "26-01-23-10-08",
  "users": [
    "Emma35",
    "Giannis",
    "Moderator"
  ],
  "moderator": "Moderator",
  "user_prompts": [
    "{\"context\": \"You are taking part in an online conversation\", \"instructions\": \"Act like a human would\", \"type\": \"1\", \"persona\": {\"username\": \"Emma35\", \"age\": 38, \"sex\": \"female\", \"sexual_orientation\": \"Heterosexual\", \"demographic_group\": \"Latino\", \"current_employment\": \"Registered Nurse\", \"education_level\": \"Bachelor's\", \"special_instructions\": \"\", \"personality_characteristics\": [\"compassionate\", \"patient\", \"diligent\", \"overwhelmed\"]}}",
    "{\"context\": \"You are taking part in an online conversation\", \"instructions\": \"Act like a human would\", \"type\": \"1\", \"persona\": {\"username\": \"Giannis\", \"age\": 21, \"sex\": \"male\", \"sexual_orientation\": \"Pansexual\", \"demographic_group\": \"White\", \"current_employment\": \"G

Congratulations, you can now run fully synthetic discussions! You may want to experiment with adding more than 2 users or testing more realistic turn taking procedures (for example, check out the `RandomWeighted` turn manager).